# XAI Air Quality Analysis - Beijing Dongsi Station

**Course**: WQF7009 Explainable Artificial Intelligence (XAI)  
**University**: University of Malaya  
**Dataset**: Beijing Multi-Site Air Quality - Dongsi Station (Urban Center)  
**Target**: PM2.5 Prediction (Regression)

---

## Table of Contents

1. [Setup & Imports](#1-setup--imports)
2. [Load Data](#2-load-data)
3. [Exploratory Data Analysis (EDA)](#3-exploratory-data-analysis-eda)
4. [Data Preprocessing](#4-data-preprocessing)
5. [Model Training (XGBoost)](#5-model-training-xgboost)
6. [Model Evaluation](#6-model-evaluation)
7. [SHAP Explanation (Feature Importance)](#7-shap-explanation-feature-importance)
8. [DiCE Counterfactual Explanation](#8-dice-counterfactual-explanation)
9. [Summary & Conclusions](#9-summary--conclusions)

---
## 1. Setup & Imports

In [ ]:
# Data processing
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb

# XAI Methods
import shap
import dice_ml

# Utilities
import joblib
import warnings
warnings.filterwarnings('ignore')

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')

print("All libraries imported successfully!")

---
## 2. Load Data

**Dataset**: Beijing Multi-Site Air Quality - Dongsi Station  
**Source**: UCI Machine Learning Repository  
**Rows**: 35,064 hourly observations (March 2013 - February 2017)

In [ ]:
# Load Dongsi station data
DATA_PATH = '../beijing-multisite-uci/raw/PRSA_Data_20130301-20170228/PRSA_Data_Dongsi_20130301-20170228.csv'

df_raw = pd.read_csv(DATA_PATH)

print(f"Dataset shape: {df_raw.shape}")
print(f"Rows: {df_raw.shape[0]:,}")
print(f"Columns: {df_raw.shape[1]}")

In [ ]:
# Display first few rows
df_raw.head(10)

In [ ]:
# Data types and info
df_raw.info()

In [ ]:
# Column descriptions
column_descriptions = {
    'No': 'Row index (to drop)',
    'year': 'Year (to combine into datetime)',
    'month': 'Month (feature)',
    'day': 'Day (to combine into datetime)',
    'hour': 'Hour (feature)',
    'PM2.5': 'TARGET - Fine particulate matter (µg/m³)',
    'PM10': 'Coarse particulate matter (µg/m³)',
    'SO2': 'Sulfur dioxide (µg/m³)',
    'NO2': 'Nitrogen dioxide (µg/m³)',
    'CO': 'Carbon monoxide (µg/m³)',
    'O3': 'Ozone (µg/m³)',
    'TEMP': 'Temperature (°C)',
    'PRES': 'Atmospheric pressure (hPa)',
    'DEWP': 'Dew point temperature (°C)',
    'RAIN': 'Precipitation (mm)',
    'wd': 'Wind direction (16 categories)',
    'WSPM': 'Wind speed (m/s)',
    'station': 'Station name (constant - Dongsi)'
}

for col, desc in column_descriptions.items():
    print(f"{col:10} : {desc}")

---
## 3. Exploratory Data Analysis (EDA)

In [ ]:
# Basic statistics
df_raw.describe()

In [ ]:
# Missing values
missing = df_raw.isnull().sum()
missing_pct = (missing / len(df_raw) * 100).round(2)
missing_df = pd.DataFrame({'Missing Count': missing, 'Missing %': missing_pct})
missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

In [ ]:
# Target variable distribution (PM2.5)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(df_raw['PM2.5'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('PM2.5 (µg/m³)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of PM2.5')
axes[0].axvline(df_raw['PM2.5'].mean(), color='red', linestyle='--', label=f"Mean: {df_raw['PM2.5'].mean():.1f}")
axes[0].legend()

# Box plot
axes[1].boxplot(df_raw['PM2.5'].dropna())
axes[1].set_ylabel('PM2.5 (µg/m³)')
axes[1].set_title('PM2.5 Box Plot')

plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/pm25_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"PM2.5 Statistics:")
print(f"  Mean: {df_raw['PM2.5'].mean():.2f} µg/m³")
print(f"  Median: {df_raw['PM2.5'].median():.2f} µg/m³")
print(f"  Std: {df_raw['PM2.5'].std():.2f} µg/m³")
print(f"  Min: {df_raw['PM2.5'].min():.2f} µg/m³")
print(f"  Max: {df_raw['PM2.5'].max():.2f} µg/m³")

In [ ]:
# Wind direction distribution
wd_counts = df_raw['wd'].value_counts()

plt.figure(figsize=(12, 5))
wd_counts.plot(kind='bar', edgecolor='black', alpha=0.7)
plt.xlabel('Wind Direction')
plt.ylabel('Count')
plt.title('Wind Direction Distribution')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/wind_direction_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nWind directions: {df_raw['wd'].nunique()} unique values")

In [ ]:
# Correlation heatmap (numerical features only)
numerical_cols = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
corr_matrix = df_raw[numerical_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='RdBu_r', center=0, 
            square=True, linewidths=0.5)
plt.title('Correlation Matrix - Air Quality Features')
plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/correlation_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# Top correlations with PM2.5
pm25_corr = corr_matrix['PM2.5'].drop('PM2.5').sort_values(key=abs, ascending=False)
print("\nCorrelation with PM2.5:")
for feat, corr in pm25_corr.items():
    print(f"  {feat:8}: {corr:+.3f}")

In [ ]:
# PM2.5 by hour of day
hourly_pm25 = df_raw.groupby('hour')['PM2.5'].mean()

plt.figure(figsize=(12, 5))
hourly_pm25.plot(kind='bar', edgecolor='black', alpha=0.7, color='steelblue')
plt.xlabel('Hour of Day')
plt.ylabel('Average PM2.5 (µg/m³)')
plt.title('Average PM2.5 by Hour of Day')
plt.xticks(rotation=0)
plt.axhline(hourly_pm25.mean(), color='red', linestyle='--', label='Overall Mean')
plt.legend()
plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/pm25_by_hour.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# PM2.5 by month
monthly_pm25 = df_raw.groupby('month')['PM2.5'].mean()

plt.figure(figsize=(12, 5))
monthly_pm25.plot(kind='bar', edgecolor='black', alpha=0.7, color='coral')
plt.xlabel('Month')
plt.ylabel('Average PM2.5 (µg/m³)')
plt.title('Average PM2.5 by Month')
plt.xticks(rotation=0)
plt.axhline(monthly_pm25.mean(), color='red', linestyle='--', label='Overall Mean')
plt.legend()
plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/pm25_by_month.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nSeasonal Pattern:")
print(f"  Winter (Dec-Feb) avg: {df_raw[df_raw['month'].isin([12, 1, 2])]['PM2.5'].mean():.1f} µg/m³")
print(f"  Summer (Jun-Aug) avg: {df_raw[df_raw['month'].isin([6, 7, 8])]['PM2.5'].mean():.1f} µg/m³")

---
## 4. Data Preprocessing

In [ ]:
# Create a copy for preprocessing
df = df_raw.copy()

print(f"Original shape: {df.shape}")

In [ ]:
# Step 1: Create datetime column
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
print(f"Date range: {df['datetime'].min()} to {df['datetime'].max()}")

In [ ]:
# Step 2: Drop unnecessary columns
columns_to_drop = ['No', 'year', 'day', 'station', 'datetime']
df = df.drop(columns=columns_to_drop)

print(f"Columns after dropping: {list(df.columns)}")
print(f"Shape: {df.shape}")

In [ ]:
# Step 3: Handle missing values
print("Missing values before:")
print(df.isnull().sum())

# Drop rows with missing target (PM2.5)
df = df.dropna(subset=['PM2.5'])

# For other features, fill with median (numerical) or mode (categorical)
numerical_features = ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
for col in numerical_features:
    df[col] = df[col].fillna(df[col].median())

# Fill missing wind direction with mode
df['wd'] = df['wd'].fillna(df['wd'].mode()[0])

print(f"\nMissing values after: {df.isnull().sum().sum()}")
print(f"Final shape: {df.shape}")

In [ ]:
# Step 4: Define features and target
TARGET = 'PM2.5'

FEATURES = ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM', 'wd', 'hour', 'month']

print(f"Target: {TARGET}")
print(f"Features ({len(FEATURES)}): {FEATURES}")

In [ ]:
# Step 5: Prepare X and y
X = df[FEATURES].copy()
y = df[TARGET].copy()

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\nFeature types:")
print(X.dtypes)

In [ ]:
# Step 6: Convert 'wd' to category type for XGBoost
X['wd'] = X['wd'].astype('category')

print(f"Wind direction categories: {X['wd'].cat.categories.tolist()}")

In [ ]:
# Step 7: Train/Validation/Test split (70/15/15)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42)  # 0.176 * 0.85 ≈ 0.15

print(f"Train set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Val set:   {X_val.shape[0]:,} samples ({X_val.shape[0]/len(X)*100:.1f}%)")
print(f"Test set:  {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")

In [ ]:
# Step 8: Save processed data
train_data = X_train.copy()
train_data[TARGET] = y_train
train_data.to_csv('../data/processed/train.csv', index=False)

val_data = X_val.copy()
val_data[TARGET] = y_val
val_data.to_csv('../data/processed/val.csv', index=False)

test_data = X_test.copy()
test_data[TARGET] = y_test
test_data.to_csv('../data/processed/test.csv', index=False)

print("Processed data saved to data/processed/")

In [ ]:
# Summary of preprocessing
print("="*60)
print("PREPROCESSING SUMMARY")
print("="*60)
print(f"Original rows: {df_raw.shape[0]:,}")
print(f"Final rows: {len(X):,}")
print(f"Rows removed: {df_raw.shape[0] - len(X):,}")
print(f"\nFeatures: {len(FEATURES)}")
print(f"  - Pollutants (5): PM10, SO2, NO2, CO, O3")
print(f"  - Weather (5): TEMP, PRES, DEWP, RAIN, WSPM")
print(f"  - Wind (1): wd (categorical)")
print(f"  - Temporal (2): hour, month")
print(f"\nTarget: {TARGET}")
print("="*60)

---
## 5. Model Training (XGBoost)

**Model**: XGBoost Regressor (black-box model)  
**Why XGBoost**:
- Handles mixed features (categorical + numerical)
- Strong baseline performance for tabular data
- Black-box nature suitable for XAI demonstration
- SHAP has optimized TreeExplainer for tree-based models

In [ ]:
# Initialize XGBoost model
model = xgb.XGBRegressor(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    enable_categorical=True,  # Enable native categorical support
    tree_method='hist'  # Required for categorical support
)

print("XGBoost model initialized with parameters:")
print(f"  n_estimators: 200")
print(f"  max_depth: 6")
print(f"  learning_rate: 0.1")
print(f"  enable_categorical: True")

In [ ]:
# Train the model
print("Training XGBoost model...")

model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    verbose=50
)

print("\nTraining complete!")

In [ ]:
# Save the model
MODEL_PATH = '../models/xgboost_model.pkl'
joblib.dump(model, MODEL_PATH)
print(f"Model saved to {MODEL_PATH}")

---
## 6. Model Evaluation

In [ ]:
# Make predictions
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

In [ ]:
# Calculate metrics
def evaluate_model(y_true, y_pred, dataset_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n{dataset_name} Performance:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  R²:   {r2:.4f}")
    
    return {'RMSE': rmse, 'MAE': mae, 'R2': r2}

train_metrics = evaluate_model(y_train, y_train_pred, "Training")
val_metrics = evaluate_model(y_val, y_val_pred, "Validation")
test_metrics = evaluate_model(y_test, y_test_pred, "Test")

In [ ]:
# Metrics summary table
metrics_df = pd.DataFrame({
    'Dataset': ['Train', 'Validation', 'Test'],
    'RMSE': [train_metrics['RMSE'], val_metrics['RMSE'], test_metrics['RMSE']],
    'MAE': [train_metrics['MAE'], val_metrics['MAE'], test_metrics['MAE']],
    'R²': [train_metrics['R2'], val_metrics['R2'], test_metrics['R2']]
})

print("\n" + "="*60)
print("MODEL PERFORMANCE SUMMARY")
print("="*60)
print(metrics_df.to_string(index=False))
print("="*60)

In [ ]:
# Actual vs Predicted plot
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

datasets = [
    (y_train, y_train_pred, 'Training'),
    (y_val, y_val_pred, 'Validation'),
    (y_test, y_test_pred, 'Test')
]

for ax, (y_true, y_pred, name) in zip(axes, datasets):
    ax.scatter(y_true, y_pred, alpha=0.3, s=10)
    ax.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
    ax.set_xlabel('Actual PM2.5')
    ax.set_ylabel('Predicted PM2.5')
    ax.set_title(f'{name} Set')
    
plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/actual_vs_predicted.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Residual plot
residuals = y_test - y_test_pred

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Residuals vs Predicted
axes[0].scatter(y_test_pred, residuals, alpha=0.3, s=10)
axes[0].axhline(y=0, color='r', linestyle='--')
axes[0].set_xlabel('Predicted PM2.5')
axes[0].set_ylabel('Residuals')
axes[0].set_title('Residuals vs Predicted')

# Residual distribution
axes[1].hist(residuals, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='r', linestyle='--')
axes[1].set_xlabel('Residuals')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Residual Distribution')

plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/residual_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Feature importance from XGBoost (built-in)
importance = model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': FEATURES,
    'Importance': importance
}).sort_values('Importance', ascending=True)

plt.figure(figsize=(10, 8))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='steelblue', edgecolor='black')
plt.xlabel('Feature Importance (Gain)')
plt.title('XGBoost Built-in Feature Importance')
plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/xgboost_feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 7. SHAP Explanation (Feature Importance)

**Method**: SHAP (SHapley Additive exPlanations)  
**Type**: Model-agnostic, Post-hoc, Global + Local  
**Question answered**: "Which features contribute to the prediction and how?"

In [ ]:
# Initialize SHAP explainer
print("Initializing SHAP TreeExplainer...")
explainer = shap.TreeExplainer(model)

# Calculate SHAP values (use a subset for faster computation)
print("Calculating SHAP values (this may take a moment)...")
X_sample = X_test.sample(n=min(1000, len(X_test)), random_state=42)
shap_values = explainer.shap_values(X_sample)

print(f"SHAP values calculated for {len(X_sample)} samples")

### 7.1 SHAP Summary Plot (Global Feature Importance)

In [ ]:
# Summary plot - bar
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_sample, plot_type="bar", show=False)
plt.title('SHAP Feature Importance (Global)')
plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/shap_summary_bar.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Summary plot - beeswarm
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X_sample, show=False)
plt.title('SHAP Summary Plot (Beeswarm)')
plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/shap_summary_beeswarm.png', dpi=150, bbox_inches='tight')
plt.show()

### 7.2 SHAP Force Plot (Local Explanation)

In [ ]:
# Initialize JavaScript for SHAP force plots
shap.initjs()

# Force plot for a single prediction (high PM2.5)
# Find an index with high PM2.5
high_pm25_idx = y_test[y_test > y_test.quantile(0.9)].index[0]
sample_idx = X_test.index.get_loc(high_pm25_idx) if high_pm25_idx in X_test.index else 0

print(f"Explaining prediction for sample with high PM2.5")
print(f"Actual PM2.5: {y_test.iloc[sample_idx]:.2f}")
print(f"Predicted PM2.5: {model.predict(X_test.iloc[[sample_idx]])[0]:.2f}")

In [ ]:
# Waterfall plot for single prediction
shap_explanation = shap.Explanation(
    values=explainer.shap_values(X_test.iloc[[sample_idx]])[0],
    base_values=explainer.expected_value,
    data=X_test.iloc[sample_idx].values,
    feature_names=FEATURES
)

plt.figure(figsize=(12, 6))
shap.waterfall_plot(shap_explanation, show=False)
plt.title('SHAP Waterfall Plot - Single Prediction (High PM2.5)')
plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/shap_waterfall.png', dpi=150, bbox_inches='tight')
plt.show()

### 7.2.2 SHAP Comparison: Peak Hour vs Night Time

In [ ]:
# Compare: Peak Hour (8 AM) vs Night Time (3 AM)
# Find samples for each scenario

# Peak hour sample (hour = 8, morning rush)
peak_hour_mask = (X_test['hour'] == 8)
peak_hour_sample = X_test[peak_hour_mask].sample(1, random_state=42)
peak_hour_idx = X_test.index.get_loc(peak_hour_sample.index[0])

# Night time sample (hour = 3, late night)
night_mask = (X_test['hour'] == 3)
night_sample = X_test[night_mask].sample(1, random_state=42)
night_idx = X_test.index.get_loc(night_sample.index[0])

# Create waterfall plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Peak Hour
shap_peak = shap.Explanation(
    values=explainer.shap_values(X_test.iloc[[peak_hour_idx]])[0],
    base_values=explainer.expected_value,
    data=X_test.iloc[peak_hour_idx].values,
    feature_names=FEATURES
)

# Night Time
shap_night = shap.Explanation(
    values=explainer.shap_values(X_test.iloc[[night_idx]])[0],
    base_values=explainer.expected_value,
    data=X_test.iloc[night_idx].values,
    feature_names=FEATURES
)

plt.subplot(1, 2, 1)
shap.waterfall_plot(shap_peak, show=False)
plt.title(f'Peak Hour (8 AM)\nActual PM2.5: {y_test.iloc[peak_hour_idx]:.1f}')

plt.subplot(1, 2, 2)
shap.waterfall_plot(shap_night, show=False)
plt.title(f'Night Time (3 AM)\nActual PM2.5: {y_test.iloc[night_idx]:.1f}')

plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/shap_peak_vs_night.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n' + '='*60)
print('COMPARISON: Peak Hour (8 AM) vs Night Time (3 AM)')
print('='*60)
print(f'Peak Hour PM2.5: {y_test.iloc[peak_hour_idx]:.1f} µg/m³')
print(f'Night Time PM2.5: {y_test.iloc[night_idx]:.1f} µg/m³')
print('='*60)

### 7.2.3 SHAP Comparison: Winter vs Summer

In [ ]:
# Compare: Winter (December) vs Summer (July)

# Winter sample (month = 12)
winter_mask = (X_test['month'] == 12)
winter_sample = X_test[winter_mask].sample(1, random_state=42)
winter_idx = X_test.index.get_loc(winter_sample.index[0])

# Summer sample (month = 7)
summer_mask = (X_test['month'] == 7)
summer_sample = X_test[summer_mask].sample(1, random_state=42)
summer_idx = X_test.index.get_loc(summer_sample.index[0])

# Create waterfall plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Winter
shap_winter = shap.Explanation(
    values=explainer.shap_values(X_test.iloc[[winter_idx]])[0],
    base_values=explainer.expected_value,
    data=X_test.iloc[winter_idx].values,
    feature_names=FEATURES
)

# Summer
shap_summer = shap.Explanation(
    values=explainer.shap_values(X_test.iloc[[summer_idx]])[0],
    base_values=explainer.expected_value,
    data=X_test.iloc[summer_idx].values,
    feature_names=FEATURES
)

plt.subplot(1, 2, 1)
shap.waterfall_plot(shap_winter, show=False)
plt.title(f'Winter (December)\nActual PM2.5: {y_test.iloc[winter_idx]:.1f}')

plt.subplot(1, 2, 2)
shap.waterfall_plot(shap_summer, show=False)
plt.title(f'Summer (July)\nActual PM2.5: {y_test.iloc[summer_idx]:.1f}')

plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/shap_winter_vs_summer.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n' + '='*60)
print('COMPARISON: Winter (December) vs Summer (July)')
print('='*60)
print(f'Winter PM2.5: {y_test.iloc[winter_idx]:.1f} µg/m³')
print(f'Summer PM2.5: {y_test.iloc[summer_idx]:.1f} µg/m³')
print('='*60)

### 7.2.4 SHAP Comparison: High vs Low Pollution

In [ ]:
# Compare: High Pollution (>90th percentile) vs Low Pollution (<10th percentile)

# High pollution sample
high_threshold = y_test.quantile(0.9)
high_mask = y_test > high_threshold
high_sample = X_test[high_mask].sample(1, random_state=42)
high_idx = X_test.index.get_loc(high_sample.index[0])

# Low pollution sample
low_threshold = y_test.quantile(0.1)
low_mask = y_test < low_threshold
low_sample = X_test[low_mask].sample(1, random_state=42)
low_idx = X_test.index.get_loc(low_sample.index[0])

# Create waterfall plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# High Pollution
shap_high = shap.Explanation(
    values=explainer.shap_values(X_test.iloc[[high_idx]])[0],
    base_values=explainer.expected_value,
    data=X_test.iloc[high_idx].values,
    feature_names=FEATURES
)

# Low Pollution
shap_low = shap.Explanation(
    values=explainer.shap_values(X_test.iloc[[low_idx]])[0],
    base_values=explainer.expected_value,
    data=X_test.iloc[low_idx].values,
    feature_names=FEATURES
)

plt.subplot(1, 2, 1)
shap.waterfall_plot(shap_high, show=False)
plt.title(f'High Pollution (>90th percentile)\nActual PM2.5: {y_test.iloc[high_idx]:.1f}')

plt.subplot(1, 2, 2)
shap.waterfall_plot(shap_low, show=False)
plt.title(f'Low Pollution (<10th percentile)\nActual PM2.5: {y_test.iloc[low_idx]:.1f}')

plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/shap_high_vs_low_pollution.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n' + '='*60)
print('COMPARISON: High Pollution vs Low Pollution')
print('='*60)
print(f'High Pollution PM2.5: {y_test.iloc[high_idx]:.1f} µg/m³ (Top 10%)')
print(f'Low Pollution PM2.5: {y_test.iloc[low_idx]:.1f} µg/m³ (Bottom 10%)')
print('='*60)

### 7.3 SHAP Dependence Plots

In [ ]:
# Dependence plots for top features
top_features = ['CO', 'PM10', 'NO2', 'TEMP']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

for ax, feature in zip(axes.flatten(), top_features):
    shap.dependence_plot(feature, shap_values, X_sample, ax=ax, show=False)
    ax.set_title(f'SHAP Dependence: {feature}')

plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/shap_dependence_plots.png', dpi=150, bbox_inches='tight')
plt.show()

### 7.4 SHAP Analysis for Wind Direction (wd)

In [ ]:
# SHAP values by wind direction
# Get the index of 'wd' feature
wd_idx = FEATURES.index('wd')
wd_shap = shap_values[:, wd_idx]

# Create DataFrame with wind direction and SHAP values
wd_shap_df = pd.DataFrame({
    'wd': X_sample['wd'].values,
    'SHAP': wd_shap
})

# Calculate mean SHAP value for each wind direction
wd_mean_shap = wd_shap_df.groupby('wd')['SHAP'].mean().sort_values()

# Plot
plt.figure(figsize=(12, 6))
colors = ['green' if x < 0 else 'red' for x in wd_mean_shap.values]
wd_mean_shap.plot(kind='barh', color=colors, edgecolor='black')
plt.xlabel('Mean SHAP Value (Impact on PM2.5)')
plt.ylabel('Wind Direction')
plt.title('SHAP: How Each Wind Direction Affects PM2.5 Prediction')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.savefig('../outputs/figures/01_shap_dice_analysis/shap_wind_direction.png', dpi=150, bbox_inches='tight')
plt.show()

print('\nWind Direction Impact on PM2.5:')
print('='*50)
print('Directions that REDUCE pollution (negative SHAP):')
for wd, val in wd_mean_shap[wd_mean_shap < 0].items():
    print(f'  {wd:5}: {val:+.2f}')
print('\nDirections that INCREASE pollution (positive SHAP):')
for wd, val in wd_mean_shap[wd_mean_shap >= 0].items():
    print(f'  {wd:5}: {val:+.2f}')

### 7.5 SHAP Summary

In [ ]:
# SHAP analysis summary
mean_shap = np.abs(shap_values).mean(axis=0)
shap_importance = pd.DataFrame({
    'Feature': FEATURES,
    'Mean |SHAP|': mean_shap
}).sort_values('Mean |SHAP|', ascending=False)

print("\n" + "="*60)
print("SHAP FEATURE IMPORTANCE SUMMARY")
print("="*60)
print(shap_importance.to_string(index=False))
print("="*60)

---
## 8. DiCE Counterfactual Explanation

**Method**: DiCE (Diverse Counterfactual Explanations)  
**Type**: Model-agnostic, Post-hoc, Local  
**Question answered**: "What minimal changes would result in a different outcome?"

In [ ]:
# Prepare data for DiCE
# DiCE requires a DataFrame with both features and target

# Create a sample dataset for DiCE
dice_data = X_train.copy()
dice_data[TARGET] = y_train

# Convert categorical to string for DiCE
dice_data['wd'] = dice_data['wd'].astype(str)

print(f"DiCE data shape: {dice_data.shape}")

In [ ]:
# Define feature types for DiCE
continuous_features = ['PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
categorical_features = ['wd', 'hour', 'month']

# Create DiCE data object
d = dice_ml.Data(
    dataframe=dice_data,
    continuous_features=continuous_features,
    outcome_name=TARGET
)

print("DiCE Data object created")

In [ ]:
# Create a wrapper model for DiCE
# DiCE converts columns to 'object' type, but XGBoost needs proper types

class ModelWrapper:
    def __init__(self, model, cat_feature='wd', cat_categories=None, continuous_features=None):
        self.model = model
        self.cat_feature = cat_feature
        self.cat_categories = cat_categories
        self.continuous_features = continuous_features or []
    
    def predict(self, X):
        X_copy = X.copy()
        
        # Convert continuous features to float (DiCE may convert them to object)
        for col in self.continuous_features:
            if col in X_copy.columns:
                X_copy[col] = pd.to_numeric(X_copy[col], errors='coerce')
        
        # Convert integer features
        for col in ['hour', 'month']:
            if col in X_copy.columns:
                X_copy[col] = pd.to_numeric(X_copy[col], errors='coerce').astype(int)
        
        # Convert categorical feature
        if self.cat_feature in X_copy.columns:
            X_copy[self.cat_feature] = pd.Categorical(
                X_copy[self.cat_feature], 
                categories=self.cat_categories
            )
        
        return self.model.predict(X_copy)

# Get categories from training data
wd_categories = X_train['wd'].cat.categories.tolist()
wrapped_model = ModelWrapper(model, 'wd', wd_categories, continuous_features)

# Create DiCE model object
m = dice_ml.Model(model=wrapped_model, backend='sklearn', model_type='regressor')

print("DiCE Model object created")

In [ ]:
# Create DiCE explainer
exp = dice_ml.Dice(d, m, method='random')

print("DiCE Explainer created")

In [ ]:
# Select a high pollution instance to explain
# Find a sample with high PM2.5 (above 90th percentile)
high_pollution_mask = y_test > y_test.quantile(0.9)
high_pollution_samples = X_test[high_pollution_mask].head(1)

# Convert to format DiCE expects
query_instance = high_pollution_samples.copy()
query_instance['wd'] = query_instance['wd'].astype(str)

print("Query instance (High Pollution):")
print(query_instance)
print(f"\nActual PM2.5: {y_test[high_pollution_samples.index[0]]:.2f}")
print(f"Predicted PM2.5: {model.predict(high_pollution_samples)[0]:.2f}")

In [ ]:
# Generate counterfactuals
# Target: reduce PM2.5 to below 50 (Good air quality)

print("Generating counterfactual explanations...")
print("Goal: Reduce PM2.5 from high (>100) to good (<50)")

cf = exp.generate_counterfactuals(
    query_instance,
    total_CFs=3,
    desired_range=[0, 50],  # Target PM2.5 range
    features_to_vary=continuous_features + ['wd'],  # Allow these features to change
    permitted_range={
        'CO': [0, 5000],
        'NO2': [0, 200],
        'WSPM': [0, 20],
        'TEMP': [-20, 40]
    }
)

print("\nCounterfactuals generated!")

In [ ]:
# Display counterfactuals
cf.visualize_as_dataframe(show_only_changes=True)

In [ ]:
# Save counterfactuals to CSV
cf_df = cf.cf_examples_list[0].final_cfs_df
cf_df.to_csv('../outputs/tables/counterfactuals.csv', index=False)
print("Counterfactuals saved to outputs/tables/counterfactuals.csv")

In [ ]:
# Interpret counterfactuals
print("\n" + "="*60)
print("COUNTERFACTUAL INTERPRETATION")
print("="*60)
print("\nOriginal Instance (High Pollution):")
print(f"  PM2.5: {y_test[high_pollution_samples.index[0]]:.1f} µg/m³ (Unhealthy)")
print("\nTo achieve Good Air Quality (PM2.5 < 50), the model suggests:")
print("-" * 60)

original = query_instance.iloc[0]
for i, cf_row in cf_df.iterrows():
    print(f"\nCounterfactual {i+1}:")
    for col in FEATURES:
        if col in cf_row.index:
            orig_val = original[col]
            cf_val = cf_row[col]
            if orig_val != cf_val:
                print(f"  {col}: {orig_val} → {cf_val}")
    print(f"  Resulting PM2.5: {cf_row[TARGET]:.1f} µg/m³")

print("="*60)

---
## 9. Summary & Conclusions

In [ ]:
print("="*70)
print("XAI AIR QUALITY ANALYSIS - SUMMARY")
print("="*70)

print("\n📊 DATASET")
print("-"*70)
print(f"  Source: Beijing Dongsi Station (Urban Center)")
print(f"  Period: March 2013 - February 2017")
print(f"  Samples: {len(X):,} hourly observations")
print(f"  Features: {len(FEATURES)} (5 pollutants + 5 weather + 1 wind + 2 temporal)")
print(f"  Target: PM2.5 (Fine Particulate Matter)")

print("\n🤖 MODEL PERFORMANCE")
print("-"*70)
print(f"  Model: XGBoost Regressor")
print(f"  Test RMSE: {test_metrics['RMSE']:.4f}")
print(f"  Test MAE: {test_metrics['MAE']:.4f}")
print(f"  Test R²: {test_metrics['R2']:.4f}")

print("\n🔍 SHAP INSIGHTS (Feature Importance)")
print("-"*70)
print("  Top 5 Important Features:")
for i, row in shap_importance.head(5).iterrows():
    print(f"    {i+1}. {row['Feature']}: {row['Mean |SHAP|']:.4f}")

print("\n🔄 COUNTERFACTUAL INSIGHTS (DiCE)")
print("-"*70)
print("  To reduce PM2.5 from Unhealthy to Good:")
print("    - Reduce CO emissions (traffic, industry)")
print("    - Increase wind speed (natural dispersion)")
print("    - Wind direction change (NW clears pollution)")

print("\n📁 OUTPUTS GENERATED")
print("-"*70)
print("  Figures:")
print("    - outputs/figures/pm25_distribution.png")
print("    - outputs/figures/correlation_matrix.png")
print("    - outputs/figures/shap_summary_bar.png")
print("    - outputs/figures/shap_summary_beeswarm.png")
print("    - outputs/figures/shap_waterfall.png")
print("    - outputs/figures/shap_dependence_plots.png")
print("  Tables:")
print("    - outputs/tables/counterfactuals.csv")
print("  Model:")
print("    - models/xgboost_model.pkl")

print("\n" + "="*70)
print("Analysis Complete!")
print("="*70)